In [199]:
#Downloading Dataset directly into the harddrive
!curl --header "Host: storage.googleapis.com" --header "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header "Accept-Language: en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7,mr;q=0.6,hi;q=0.5,pa;q=0.4" --header "Referer: https://eightportions.com/datasets/Recipes/" "https://storage.googleapis.com/recipe-box/recipes_raw.zip" -o "recipes_raw.zip" -L

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 50.8M  100 50.8M    0     0  37.3M      0  0:00:01  0:00:01 --:--:-- 37.3M


In [0]:
#Unpack the zip file
import shutil
shutil.unpack_archive('/content/recipes_raw.zip')

In [0]:
import requests 
import urllib3
from bs4 import BeautifulSoup
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
import re

In [0]:
import pandas as pd
import json

df=pd.read_json('/content/recipes_raw_nosource_epi.json',orient='frame').drop(['picture_link'])

In [0]:
df=df.transpose()[['ingredients','title']]

In [285]:
df.reset_index(drop='True').ingredients[1]

['18 fresh chestnuts',
 '2 1/2 pounds veal stew meat, cut into 2x1-inch pieces',
 '4 tablespoons olive oil',
 '1 1/2 cups chopped onion',
 '1 1/2 tablespoons chopped garlic',
 '1 bay leaf',
 '2 1/2 cups canned low-salt chicken broth',
 '3/4 cup dry white wine',
 '6 medium carrots, peeled, cut into 1-inch pieces',
 '3 tablespoons chopped fresh sage']

In [0]:
df=df.reset_index(drop='True')

In [287]:
df.head(10)

,ingredients,title
0,"[12 egg whites, 12 egg yolks, 1 1/2 cups sugar...",Christmas Eggnog
1,"[18 fresh chestnuts, 2 1/2 pounds veal stew me...","Veal, Carrot and Chestnut Ragoût"
2,"[2 tablespoons unsalted butter, softened, 4 or...",Caramelized Bread Pudding with Chocolate and C...
3,"[3/4 pound Stilton, crumbled (about 3 cups) an...",Sherried Stilton and Green Peppercorn Spread
4,"[2 cups (about 9 1/2 ounces) whole almonds, to...",Almond-Chocolate Macaroons
5,"[2 tablespoons butter, 2 tablespoons flour, 1 ...",White Sauce or Bechamel Sauce
6,"[1/2 teaspoon finely grated fresh lemon zest, ...","Grilled Whole Mackerel with Lemon, Oregano, an..."
7,"[2 garlic cloves, finely chopped, 2 teaspoons ...",Apricot Chutney
8,"[2 tablespoons (1/4 stick) butter, 1/4 cup oli...",Garlic Croutons
9,"[1 recipe pâte à chou, About 1 cup pastry crea...",Christmas Croquembouche


In [288]:
df.ingredients.values[0]

['12 egg whites',
 '12 egg yolks',
 '1 1/2 cups sugar',
 '3/4 cup rye whiskey',
 '12 egg whites',
 '3/4 cup brandy',
 '1/2 cup rum',
 '1 to 2 cups heavy cream, lightly whipped',
 'Garnish: ground nutmeg']

**BASIC PREPROCESSING**

In [0]:

stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [0]:
ingridients=[]
for item in df.ingredients.values:
  x= " ".join(item)
  x = re.sub("^\d+\s|\s\d+\s|\s\d+$|[^\w]", " ", x)
  x = re.sub(r'[0-9]+', ' ',x)
  x = re.sub(r' +',' ',x)
  x =  x.split(" ")
  x=list(set(x))[1:]
  x=[i for i in x if i not in stop_words]
  ingridients.append(x)


#**WEB SCRAPPING FOR FOOD LIST**

***As we have alot of unncessary words in the the recipe(cups,tablesspoons,number etc).We need to remove them and only have the words which are foods.For this we need to know which words are actual foods so we webscrap these words from various websites and create a list which we will use to filter our ingredient later*** 

In [0]:
def preprocess(items):
      items=" ".join(items)
      items = re.sub("[^\w]", " ",items)
      items= re.sub(r' +',' ',items).lower().split(" ")
      items=list(set(items))
      return items


In [0]:
#https://www.geeksforgeeks.org/implementing-web-scraping-python-beautiful-soup/

#We scrap 

import requests
from bs4 import BeautifulSoup 
import csv 
URL = "http://www.thefoodcoach.com.au/food/?Alpha=%5BA-Z%5D"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
  
table = soup.find('div', attrs = {'class':'search_results mb-5'}) 
items=[] 
table_2=table.find_all('td')
for row in table_2:
    items.append(row.a.text.rstrip())

In [0]:
items=preprocess(items)

In [0]:
URL = "https://www.enchantedlearning.com/wordlist/food.shtml"
r = requests.get(URL) 
item_2=[] 
soup = BeautifulSoup(r.content, 'html5lib')   
table = soup.find_all('div', attrs = {'wordlist-item'}) 
for row in table:
  item_2.append(row.text)
  

In [0]:
item_2=preprocess(item_2)

In [0]:
URL = "https://foodforbreastcancer.com/food-list.php"
r = requests.get(URL) 
items_3=[]
soup = BeautifulSoup(r.content, 'html5lib')
table = soup.find_all('li') 
for row in table:
  items_3.append(row.text)

In [0]:
items_3=preprocess(items_3)

In [0]:
#We scrap 3 websites and get 3 lists of food items which we merge by eliminating duplicated using the set operations.

full_list=list(set(list(set(items+item_2))+items_3))

In [375]:
len(full_list)

1107

# **PORTER STEMMING ALL THE WORDS**

***We need to make sure that all of our words are resistant to plural,singular variance and thus we convert each word into its root word using porter stemming.***

In [0]:
def stem(a):
    p = nltk.PorterStemmer()
    a = [p.stem(word) for word in a] # ['s', 'o', 'c', ',', ' ', '3', '2', '\n', ...]
    return a

In [0]:
filtered_ingridients=[]
for row in ingridients:
  row=stem(row)
  filtered_ingridients.append(row)

In [0]:
filtered_items=stem(full_list)

# **FILTERING OUR INGRIDIENTS**

***Now that we have stemmed our words into the root words it is time to filter the main recipe where only the food words will left and nothing else***

In [0]:
main_ingridients=[]
for row in filtered_ingridients:
  row=set(row)
  items=set(filtered_items)
  new_row=row.intersection(items)
  main_ingridients.append(list(new_row))

# **FINAL DATAFRAME WITH DISHES AND THERE INGRIDIENTS**

In [0]:
recipe=[",".join(x) for x in main_ingridients]

In [0]:
df.ingredients=recipe

In [484]:
df

,ingredients,title
0,"egg,nutmeg,whip,rye,sugar,cream,yolk,white",Christmas Eggnog
1,"low,bay,white,oliv,wine,stew,veal,sage,chestnu...","Veal, Carrot and Chestnut Ragoût"
2,"extra,brown,good,chop,salt,white,bread,nutmeg,...",Caramelized Bread Pudding with Chocolate and C...
3,"chees,peppercorn,cream,green,dri",Sherried Stilton and Green Peppercorn Spread
4,"white,cinnamon,egg,extract,almond,sugar,toast,...",Almond-Chocolate Macaroons
...,...,...
25318,"pepper,sauc,oliv,celeri,bell,lentil,oil,beef,g...",Spinach and Lentil Soup
25319,"coriand,water,powder,spinach,chop,coars,oil,in...",Fresh Cheese with Spinach
25320,"tortilla,water,black,chop,salt,leav,oliv,coars...",Jalapeño and Lime–Marinated Skirt Steak Tacos
25321,"egg,bake,butter,semolina,almond,juic,sugar,pow...",Semolina–Lemon Syrup Cakes


# **USING WORD2VEC**

***We vectorize our ingredients per dish to find some similarity later using various distances***

In [0]:

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [0]:
w2v_model_train = Word2Vec(main_ingridients, min_count = 1, size =150, workers= 4)

In [314]:
w2v_words_train = list(w2v_model_train.wv.vocab)
print('number of words that occured minimum 5 times ',len(w2v_words_train))
print('sample words', w2v_words_train[0:10])

number of words that occured minimum 5 times  824
sample words ['egg', 'nutmeg', 'whip', 'rye', 'sugar', 'cream', 'yolk', 'white', 'low', 'bay']


In [0]:
from tqdm import tqdm

In [316]:
train_w2v = []

for sent in tqdm(main_ingridients):
    sent_vect = np.zeros(150)
    cnt_words = 0
    for word in sent:
        if word in w2v_words_train:
            vect = w2v_model_train.wv[word]#creating a matrix of 50 length for each word
            sent_vect += vect 
            cnt_words = cnt_words + 1
    if cnt_words !=0:
        sent_vect /= cnt_words
    
        train_w2v.append(sent_vect)

100%|██████████| 25323/25323 [00:02<00:00, 9305.53it/s]


In [0]:
train_w2v=np.array(train_w2v)

**USING COSINE-SIMILARITY**

In [0]:

from sklearn.metrics.pairwise import euclidean_distances

x=euclidean_distances(train_w2v)

In [0]:

def similar_word(index):
    
    row=x[index]
    max_cosine=np.argsort(row)[:5]
    return df.iloc[max_cosine]

In [485]:
print(df.iloc[0].ingredients)

egg,nutmeg,whip,rye,sugar,cream,yolk,white


In [486]:

for m in similar_word(0).ingredients.values:
  print(m)


egg,nutmeg,whip,rye,sugar,cream,yolk,white
coriand,eye,black,chop,salt,leav,smoke,cream,oil,slice,garlic,tomatillo,seed,cumin,cilantro,clove,ham,pepper,veget,onion,sour,chive,pea
low,eye,free,water,fat,spinach,black,salt,chicken,oliv,pea,juic,oil,mint,lemon,cayenn,fennel,coffe,seed,spice,babi,pepper,honey,orang,fresh
cayenn,coars,cinnamon,orang,almond,date,pistachio,cream,toast,dri,apricot,whole,chop,cardamom
granni,coars,appl,wine,spoon,cook,see,grape,sugar,green,powder,mint,vitamin,dri,chop,smith,leav,white


**OBSERVATIONS:**
1.We can't notice any semantic simialrity here and the results are not good.

# **USING BAG OF WORDS**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vectorizer=CountVectorizer()
vectors=vectorizer.fit_transform(df.ingredients)


In [0]:
vectors=vectors.todense()

**USING BAG OF WORDS WITH HAMMING DISTANCE**

In [0]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import hamming


#constructing a matrix where there is pairwise simialrity between all the points

hamming_matrix=pairwise_distances(vectors,metric='hamming')

In [0]:
def hamming_similarity(index):
    
    row=hamming_matrix[index]
    max_cosine=np.argsort(row)[1:6]
    return df.iloc[max_cosine]

In [494]:
print(df.iloc[1].ingredients)

low,bay,white,oliv,wine,stew,veal,sage,chestnut,oil,carrot,meat,dri,onion,chop,salt,garlic,can,chicken,fresh


In [495]:
for j in hamming_similarity(1
                            ).ingredients.values:
  print(j)

low,white,oliv,wine,fennel,seed,oil,dri,onion,chop,saffron,salt,garlic,can,chicken,fresh
low,white,oliv,wine,oil,rosemari,dri,clove,salt,garlic,can,chicken,fresh
low,bay,chicken,wine,can,butter,rosemari,dri,onion,chop,salt,leav,white,fresh
low,white,oliv,potato,wine,butter,oil,dri,salt,garlic,can,chicken
low,white,oliv,wine,celeri,sage,turkey,butter,thyme,oil,carrot,rosemari,dri,onion,flour,chop,salt,garlic,can,chicken


**OBSERVATIONS**

1.We see much better results and there is clear semantic realtion as hamming distance works really well for categroical data.The Hamming distance between 1-D arrays u and v, is simply the proportion of disagreeing components in u and v so lesser the distance more the similarity

**USING JACCARD DISTANCES FOR SIMILARITY**

In [329]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import hamming

#constructing a matrix where there is pairwise simialrity between all the points
jaccard_matrix=pairwise_distances(vectors,metric='jaccard')

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/pairwise.py:1575: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [0]:
def jaccard_similarity(index):
    
    row=jaccard_matrix[index]
    max_cosine=np.argsort(row)[1:6]
    return df.iloc[max_cosine]

In [491]:
print(df.iloc[1].ingredients)

low,bay,white,oliv,wine,stew,veal,sage,chestnut,oil,carrot,meat,dri,onion,chop,salt,garlic,can,chicken,fresh


In [492]:
for i in jaccard_similarity(1).ingredients.values:
  print(i)

low,white,oliv,wine,fennel,seed,oil,dri,onion,chop,saffron,salt,garlic,can,chicken,fresh
low,chop,salt,chicken,white,oliv,coars,wine,sage,oil,carrot,celeri,slice,garlic,order,rosemari,clove,veal,parsley,dri,onion,fresh
low,white,oliv,wine,celeri,sage,turkey,butter,thyme,oil,carrot,rosemari,dri,onion,flour,chop,salt,garlic,can,chicken
low,chicken,oliv,wine,mustard,marjoram,dijon,thyme,oil,rosemari,dri,onion,chop,salt,garlic,can,white,fresh
low,white,oliv,wine,oil,rosemari,dri,clove,salt,garlic,can,chicken,fresh


**OBSERVATIONS:**

1.Jaccard distance works very well

# **FINAL PIPELINE**

In [0]:
from scipy.spatial import distance
def find_top_5_ham(vector_i):
  jaccard_distances=[]
  for vector_j in vectors:
       vector_i=vector_i.ravel()
       vector_j=vector_j.ravel()
       jaccard_distances.append(distance.hamming(vector_i,vector_j))
  top_5_index=np.argsort(jaccard_distances)[:5]
  top_5_df=df.iloc[top_5_index].reset_index(drop=True)
  return top_5_df

In [0]:
from scipy.spatial import distance
def find_top_5_jac(vector_i):
  jaccard_distances=[]
  for vector_j in vectors:
       vector_i=vector_i.ravel()
       vector_j=vector_j.ravel()
       jaccard_distances.append(distance.jaccard(vector_i,vector_j))
  top_5_index=np.argsort(jaccard_distances)[:5]
  top_5_df=df.iloc[top_5_index].reset_index(drop=True)
  return top_5_df

In [0]:
def run_app():
  ing=input("Enter your favourite ingridients : ")
  print()
  ing=ing.split(",")
  ing=stem(ing)
  ing=",".join(ing)
  ing_vector=vectorizer.transform([ing]).todense()
  my_top_5=find_top_5_jac(ing_vector)
  print(my_top_5.to_string())

**RUNNING WITH HAMMING DISTANCE**

In [517]:
run_app()

Enter your favourite ingridients : chicken,tomato,garlic,carrot

          ingredients                            title
0              tomato                Crushed Tomatoes 
1                                Lacy Parmesan Crisps 
2  buttermilk,chicken      Buttermilk Roasted Chicken 
3                      Grandma Reggie's chopped liver 
4                               Strozzapreti and Pici 


RUNNING WITH JACCARD DISTANCE

In [519]:
run_app()

Enter your favourite ingridients : chicken,tomato,garlic,carrot

                                         ingredients                                              title
0   basil,leav,tomato,clove,garlic,can,chicken,fresh                                Light Tomato Sauce 
1  low,butter,cream,carrot,clove,salt,garlic,chicken                                      Carrot Purée 
2                       oliv,oil,tomato,clove,garlic                             Heirloom Tomato Salad 
3  bread,oliv,organ,oil,tomato,loaf,onion,clove,g...                                 Pappa al Pomodoro 
4  bread,oil,tomato,french,dri,clove,fresh,garlic...  Chicken Breasts with Sun-Dried Tomato and Garl...


# **CONCLUSION**

1.During production we see better results with Jaccard distance than hamming distance 